In [2]:
#! /usr/bin/env python

# PonyGE2
# Copyright (c) 2017 Michael Fenton, James McDermott,
#                    David Fagan, Stefan Forstenlechner,
#                    and Erik Hemberg
# Hereby licensed under the GNU GPL v3.
""" Python GE implementation """

from utilities.algorithm.general import check_python_version

check_python_version()

import matplotlib.pyplot as plt

from stats.stats import get_stats
from algorithm.parameters import params, set_params
import sys
import time
import numpy as np
import random
import os
import pickle

from utilities.stats import trackers


In [3]:

def clear_trackers():
    """Utilities for tracking progress of runs, including time taken per
    generation, fitness plots, fitness caches, etc."""
    trackers.cache = {}
    trackers.runtime_error_cache = []
    trackers.best_fitness_list = []
    trackers.best_test_fitness_list = []
    trackers.first_pareto_list = []
    #trackers.time_list = []
    trackers.stats_list = []
    trackers.best_ever = None
    # Store the best ever individual here.

def save_data_pickle(file_path, test_results, train_results, test_evolution, train_evolution):
    data = {
        'test_results': test_results,
        'train_results': train_results,
        'test_evolution': test_evolution,
        'train_evolution': train_evolution,
    }

    with open(file_path, 'wb') as f:
        pickle.dump(data, f)

def load_data_pickle(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)

    return data['test_results'], data['train_results'], data['test_evolution'], data['train_evolution']

def save_plot(file_path, data):
        plt.plot(data)

        # Add title and labels
        plt.title('Line Plot of Values')
        plt.xlabel('Index')
        plt.ylabel('Value')

        # Display the plot
        plt.savefig(file_path)

def append_to_file(file_path, float_value, recreate = False):
    if recreate and os.path.exists(file_path):
        os.remove(file_path)
    # Open or create the file in append mode
    with open(file_path, 'a') as file:
        # Write the float value to the file, converting it to a string first
        file.write(str(float_value) + "\n")  # The "\n" adds a newline character after the float value


In [4]:

def mane():
    """ Run program """
    set_params(sys.argv[1:])  # exclude the ponyge.py arg itself

    test_results = []
    train_results = []

    test_evolution = []
    train_evolution = []

    #seeds = [12345, 54321]

    #seeds = np.random.randint(1, 4294967295, 5)
    seeds = [12345]


    if len(seeds) == 1:
        random.seed(seeds[0])
        individuals = params['SEARCH_LOOP']()

        # Print final review
        get_stats(individuals, end=True)

        # print(((trackers.best_fitness_list)))
        # print(((trackers.best_test_fitness_list)))
        save_plot('line_plot.png', trackers.best_test_fitness_list)

    else:
        crossover_rates = np.linspace(0, 1, 3)

        results = {}
        for c in crossover_rates:
            for i, seed in enumerate(seeds):
                params['ITERATION_INDEX'] = i
                params['MUTATION_PROBABILITY'] = c

                random.seed(int(seed))
                # Run evolution
                individuals = params['SEARCH_LOOP']()
                # Print final review
                get_stats(individuals, end=True)

                key = (i, c)
                results[key] = {
                    'test_result': trackers.best_ever.test_fitness,
                    'train_result': trackers.best_ever.training_fitness,
                    'test_evolution': trackers.best_test_fitness_list,
                    'train_evolution': trackers.best_fitness_list
                }

                clear_trackers()

        fitness_shape = (len(seeds), len(crossover_rates))
        fitness_list_shapes = (*fitness_shape, params['GENERATIONS'] + 1)

        test_results = np.reshape([results[key]['test_result'] for key in sorted(results)], fitness_shape)
        train_results = np.reshape([results[key]['train_result'] for key in sorted(results)], fitness_shape)
        test_evolution = np.reshape([results[key]['test_evolution'] for key in sorted(results)], fitness_list_shapes)
        train_evolution = np.reshape([results[key]['train_evolution'] for key in sorted(results)], fitness_list_shapes)

        save_data_pickle("data_results.pkl", test_results, train_results, test_evolution, train_evolution)

In [6]:
if __name__ == "__main__":
    mane()
    data = load_data_pickle("data_results.pkl")
    print((data[3].shape))

Exception: algorithm.parameters.set_params
Error: unknown parameters: ['--ip=127.0.0.1', '--stdin=9003', '--control=9001', '--hb=9000', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"72ffb3d7-25d5-40e3-8e72-7a7f7d7e68d4"', '--shell=9002', '--transport="tcp"', '--iopub=9004']
You may wish to check the spelling, add code to recognise this parameter, or use --extra_parameters